# <font color="NAVY">DS4A </font> 

<font color="NAVY">Hector</font>  
<font color="NAVY">Ximena Rios</font>  
<font color="NAVY">Proyecto e_commerce</font>  


## ¿Qué aspectos del comportamiento en línea de los consumidores son útiles para que las empresas comprendan mejor a sus clientes y predigan las tendencias del consumidor, los hábitos de gasto y las variables relacionadas con las compras?

In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy

# Introduccion 

**Tema:** Comportamiento del consumidor en el comercio electrónico

**Pregunta de investigación** ¿Qué aspectos del comportamiento en línea de los consumidores son útiles para que las empresas comprendan mejor a sus clientes y predigan las tendencias del consumidor, los hábitos de gasto y las variables relacionadas con las compras?

**Objetivos**

- Comprensión y limpieza de datos
- Análisis y modelado
- Construye modelos de predicción
- Cree paneles para visualizar la información


## Reading in the data files

Cargamos las bases de datos que vamos a analizar.

In [3]:
ruta= os.getcwd( ) #Directorio raiz de trabajo

In [43]:
#carga de dataframe
#Amazon
df_ab=df_amazon_beauty=pd.read_csv(ruta+"/data/amazon_reviews/beauty_metadata.csv") 
#Brazilian
df_bc=df_brazilian_customers=pd.read_csv(ruta+"/data/brazilian-ecommerce/olist_customers_dataset.csv") 
df_bg=df_brazilian_geolocation=pd.read_csv(ruta+"/data/brazilian-ecommerce/olist_geolocation_dataset.csv") 
df_bi=df_brazilian_items=pd.read_csv(ruta+"/data/brazilian-ecommerce/olist_order_items_dataset.csv") 
df_bop=df_brazilian_order_payments=pd.read_csv(ruta+"/data/brazilian-ecommerce/olist_order_payments_dataset.csv") 
df_bor=df_brazilian_order_reviews=pd.read_csv(ruta+"/data/brazilian-ecommerce/olist_order_reviews_dataset.csv")
df_bo=df_brazilian_orders=pd.read_csv(ruta+"/data/brazilian-ecommerce/olist_orders_dataset.csv")
df_bp=df_brazilian_products=pd.read_csv(ruta+"/data/brazilian-ecommerce/olist_products_dataset.csv")
df_bs=df_brazilian_sellers=pd.read_csv(ruta+"/data/brazilian-ecommerce/olist_sellers_dataset.csv")
df_bpc=df_brazilian_product_category=pd.read_csv(ruta+"/data/brazilian-ecommerce/product_category_name_translation.csv")
#online
df_oc=df_online_category=pd.read_csv(ruta+"/data/online_consumer_behavior/category_tree.csv")
df_oe=df_online_events=pd.read_csv(ruta+"/data/online_consumer_behavior/events.csv")
df_oi=df_online_item_properties=pd.read_csv(ruta+"/data/online_consumer_behavior/item_properties.csv")
#retailer
df_rp=df_retailer_pricing=pd.read_csv(ruta+"/data/retailer_pricing.csv")
#UK_retail
df_UK=df_UK_retail_transactions=pd.read_csv(ruta+"/data/UK_retail_transactions.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## <font color="deeppink"> retailer_pricing </font>

Las columnas presentadas en este archivo son:

|CAMPO   |TIPO   |DESCRIPCION   |
|---|---|---|
|store|STRING|El minorista bajo el cual se incluye este producto*|
|title| STRING| El título del producto.|
|sku |STRING |La unidad de existencias específica del minorista (SKU).|
|groupid |STRING|El SKU a nivel de grupo, identifica la lista de productos iguales o similares en diferentes minoristas|
|brand |STRING |La marca.|
|color |STRING |El color.|
|sizing |STRING |La talla.|
|category |STRING |La categoria.|
|subcategory |STRING |La Subcategoria.|
|price |FLOAT |El precio del producto, en USD.|
|date |STRING |La marca de tiempo para cuando se registraron los detalles (formato:yyyy-mm-dd).|


*minoristas: (0=amazon.com, 1=zappos.com, 2=macys.com, 3=neimanmarcus.com, 4=saksfifthavenue.com, 5=bloomingdales.com)

In [44]:
df_rp.head(3)

,store,title,sku,groupid,brand,color,sizing,category,subcategory,price,date
0,0,HABA My First Ball Track 44-Piece Basic Pack F...,B06W5B4CMT,5473635,HABA,_no_variant_,one size,Toys/Games,Other,119.99,20190801
1,0,Fendi Logo Zip Around Leather Wallet,B07DNHSTWR,5482941,FENDI,_no_variant_,one size,Small Leather Goods,Wallets,650.00,20190801
2,0,Hanro 'Touch Feeling' Crop Top,B00009Z3N4,517719,HANRO,white,large,Underwear/Lingerie,Bra,86.00,20190801


In [45]:
df_rp.shape

(22550375, 11)

In [46]:
df_rp.nunique()

store               6
title           68556
sku            300427
groupid         68282
brand            2094
color           54403
sizing           2646
category           39
subcategory       287
price           45473
date              216
dtype: int64

In [47]:
df_rp.isnull().sum()

store                0
title                0
sku                 23
groupid              6
brand                0
color             3197
sizing              20
category             0
subcategory    1072345
price                0
date                 0
dtype: int64

In [48]:
df_rp['store'].unique()

array([0, 1, 2, 3, 4, 5])

## <font color="deeppink"> amazon_beauty_reviews </font>

Las columnas presentadas en este archivo son:

|CAMPO   |TIPO   |DESCRIPCION   |
|---|---|---|
|title |STRING |El nombre del producto|
brand |STRING |El nombre de la marca.
rank |FLOAT |El rango del producto en la categoría Belleza.
asin |STRING |El ID del producto
description  |LIST |La descripción del producto.
also_view |LIST |Productos relacionados que vieron otros clientes.
also_buy |LIST |Productos relacionados que compraron otros clientes.
price |FLOAT |El precio del producto en USD
similar_item |LIST |Tabla de productos similares.

In [49]:
df_ab.head(3)

,title,brand,rank,asin,description,also_view,also_buy,price,similar_item
0,Workout Headphones by Arena Essentials,HarperCollins,3235148.0,0061073717,NaN,NaN,NaN,NaN,NaN
1,Black Diamond,Swedish Beauty,1462563.0,0143026860,['This is the NEW PURPLE BOTTLE that just came...,['B000LXTNMW'],NaN,NaN,NaN
2,"12 Pc BLUE HEAVEN KAJAL""HERBAL kajal With VITA...",BLUE HEAVE,100425.0,014789302X,"['A brand-new, unused, unopened, undamaged ite...","['B00OWT3W28', 'B0796RMSV8', 'B010E1X15K', 'B0...","['B00OWT3W28', 'B010E1X15K', 'B0796RMSV8', 'B0...",6.45,NaN


In [50]:
df_ab.shape

(32711, 9)

In [51]:
df_ab.nunique()

title           32112
brand            7819
rank            31763
asin            32307
description     13602
also_view        7835
also_buy         6465
price            3509
similar_item     1301
dtype: int64

## <font color="deeppink"> UK_retail_transactions </font>

Conjunto de datos de transacciones de un minorista en línea con sede en el Reino Unido, de 2009 a 2011. La empresa principalmente vende artículos de regalo únicos para todas las ocasiones. Muchos clientes de la empresa son mayoristas.

~ 1 millón de filas y 8 columnas. Tamaño: ~ 15 MB comprimido, ~ 100 MB descomprimido. 

In [1]:
df.info

NameError: name 'df' is not defined